# Read HDF files from results database
This notebook shows how to extract results from hdf (`.h5`) files that are already present in the `\results_database\` folder that is present in each benchmark folder (in `\models\`). These files can store either experimental or computational results of the same experiment/simulation. The `\results_database\` folder of the should have all the same structure and same ways to extract the data (e.g. the name of the tally).

For this tutorial notebook the `\results_database\` folder is replaced with the `\example_database\` folder. The `\example_database\` folder contains some of the actual results from the `fns_duct` benchmark as example.

In [1]:
import openmc_fusion_benchmarks as ofb

First we need to instantiate a `ResultsFromDatabase` object by providing the path to the file, including the name (`file`).

In [2]:
experiment_file = ofb.ResultsFromDatabase(file='example_database/experiment.h5')
mcnp_fendl2_file = ofb.ResultsFromDatabase(file='example_database/mcnp-4b-c_fendl2.h5')

Usually the file is read from a **postprocessing** notebook that is in the benchmark/model folder with contains the `\results_database\` folder. Therefore, the `path` argument defaults to a `\results_database\` folder.

### Extract info
If present, it is possible to extract and print some information regarding the experiment/simulation that produced the results present in the hdf file. Typical information can be:
- `when` the experiment/simulation have been performed
- `Where` the experiment/simulation have been performed (either place or institution)
- `literature_info` regarding the experiment/model/simulation (most likely the link to a paper or report)
- `code_version` if the hdf file refers to simulation results, useful info for V&V purposes
- `xs_library` (nuclear data library name and version) if the hdf file refers to simulation results, useful info for V&V purposes

Example of experimental information extraction:

In [3]:
when = experiment_file.when
where = experiment_file.where
literature = experiment_file.literature_info

print(f'When: {when}')
print(f'Where: {where}')
print(f'Literature: {literature}')

When: ['2004']
Where: ['JAERI, Tokai-mura (Japan)']
Literature: ['https://www.tandfonline.com/doi/pdf/10.1080/00223131.2004.10875640']


It is also possible to print all the information at once with the `print_all_info` method:

In [4]:
experiment_file.print_all_info()

Info:
 When: ['2004']
 Where: ['JAERI, Tokai-mura (Japan)']
 Code version: n/a
 XS library: n/a
 Literature: ['https://www.tandfonline.com/doi/pdf/10.1080/00223131.2004.10875640']



Similarly, for hdf files containing numerical simulation results:

In [5]:
cv = mcnp_fendl2_file.code_version
xslib = mcnp_fendl2_file.xs_library
literature = mcnp_fendl2_file.literature_info

mcnp_fendl2_file.print_all_info()

Info:
 When: n/a
 Where: n/a
 Code version: MCNP-4B/C
 XS library: FENDL-2.0
 Literature: ['https://www.tandfonline.com/doi/pdf/10.1080/00223131.2004.10875640']



In the case of hdf file containing simulation results it is possible to print just the `code_info`. Namely the `code_version` and `xs_library`:

In [6]:
mcnp_fendl2_file.print_code_info()

Code version: MCNP-4B/C
 XS library: FENDL-2.0



### Extract a tally

In order to extract a tally from a hdf file it is necessary to know its exact name. The `ResultsFromDatabase` object has a `list_tallies()` method that helps identifying the names of all the tallies available:

In [7]:
experiment_file.list_tallies()

<KeysViewHDF5 ['nspectrum_detector3', 'nspectrum_detector5', 'nspectrum_detector7', 'nspectrum_detector9', 'rr_au197', 'rr_in115', 'rr_nb93']>


We chose, as example, the tally named `rr_offaxis_al27` and extract its results in a Pandas DataFrame format with the `get_tally_dataframe` method:

In [8]:
results = experiment_file.get_tally_dataframe(tally_name='rr_nb93')

results

,Detector No.,mean,std. dev.
0,1,1.727746e-06,5.010465e-08
1,2,1.063465e-06,4.147514e-08
2,3,6.911705e-07,2.557331e-08
3,4,3.645722e-09,1.677032e-10
4,5,7.896895e-10,4.659168e-11
5,6,NaN,NaN
6,7,NaN,NaN
7,8,NaN,NaN
8,9,3.541471e-10,3.895618e-12
9,10,NaN,NaN


It is possible to view single columns of the dataframe by calling the column name (names listed in the bottom line of the previous):

In [9]:
print(results['mean'])
print(results['std. dev.'])

0     1.727746e-06
1     1.063465e-06
2     6.911705e-07
3     3.645722e-09
4     7.896895e-10
5              NaN
6              NaN
7              NaN
8     3.541471e-10
9              NaN
10             NaN
Name: mean, dtype: float64
0     5.010465e-08
1     4.147514e-08
2     2.557331e-08
3     1.677032e-10
4     4.659168e-11
5              NaN
6              NaN
7              NaN
8     3.895618e-12
9              NaN
10             NaN
Name: std. dev., dtype: float64


Typical experimental or statistical results (e.g. Monte Carlo simulation) are provided with the `mean` and `std. dev.` columns.
Where the `std. dev.` column provides the **standard deviation absolute values (not relative)**.